### The Golal is to Genarate a ReportCard for Trade Setup -- 20SEP25:09:36:00

In [ ]:
import pandas as pd
import mplfinance as mpf

In [ ]:
ohlc = pd.read_csv("../Src/xauusdm15.filtered.csv")

ohlc['DateTime'] = pd.to_datetime(ohlc['DateTime'])
ohlc_15M = ohlc.set_index('DateTime')


print(ohlc_15M.head(3))

In [ ]:
ohlc_4H = ohlc_15M.resample("4h").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume": "sum"
}).dropna()

print(ohlc_4H.head(5))


In [ ]:
import logging
import pandas as pd
import mplfinance as mpf
from typing import List, Optional, Dict, Any, Union
from uuid import uuid4
import numpy as np
from datetime import datetime

# Configure logging for enterprise use
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

class Chartter:
    """Enterprise-level financial charting utility using mplfinance."""

    def __init__(self, config: Optional[Dict[str, Any]] = None):
        """
        Initialize Chartter with default or custom configuration.

        Args:
            config (Dict[str, Any], optional): Custom configuration for charting.
        """
        self._default_config = {
            'chart_type': 'candle',
            'style': 'charles',
            'title': 'Financial Chart',
            'ylabel': 'Price',
            'ylabel_lower': 'Volume',
            'xlabel': 'Date',
            'figratio': (16, 5),
            'panel_ratios': (3, 1),
            'datetime_format': '%Y-%m-%d %H:%M',
            'xrotation': 20,
            'volume': True,
            'volume_panel': 2,
            'warn_too_much_data': 10000,
            'width_config': {
                'candle_linewidth': 1.0,
                'candle_width': 0.6,
                'ohlc_linewidth': 1.0,
                'volume_linewidth': 1.0
            },
            'additional_indicators': []  # e.g., ['sma', 'rsi', 'bollinger']
        }
        self.config = self._default_config if config is None else {**self._default_config, **config}
        self._validate_config()

    def _validate_config(self) -> None:
        """Validate configuration parameters."""
        valid_chart_types = ['candle', 'ohlc', 'line', 'renko', 'pnf']
        if self.config['chart_type'] not in valid_chart_types:
            logger.error(f"Invalid chart_type: {self.config['chart_type']}. Must be one of {valid_chart_types}")
            raise ValueError(f"chart_type must be one of {valid_chart_types}")

        if not isinstance(self.config['figratio'], tuple) or len(self.config['figratio']) != 2:
            logger.error("figratio must be a tuple of (width, height)")
            raise ValueError("figratio must be a tuple of (width, height)")

    def _validate_dataframe(self, df: pd.DataFrame) -> None:
        """Validate input DataFrame for required OHLCV columns and datetime index."""
        if not isinstance(df, pd.DataFrame):
            logger.error("Input must be a pandas DataFrame")
            raise ValueError("Input must be a pandas DataFrame")

        required_columns = ['Open', 'High', 'Low', 'Close']
        if not all(col in df.columns for col in required_columns):
            logger.error(f"DataFrame must contain columns: {required_columns}")
            raise ValueError(f"DataFrame must contain columns: {required_columns}")

        if not pd.api.types.is_datetime64_any_dtype(df.index):
            logger.error("DataFrame index must be datetime")
            raise ValueError("DataFrame index must be datetime")

    def _compute_indicators(self, df: pd.DataFrame) -> List[Dict]:
        """Compute additional financial indicators based on config."""
        apds = []
        for indicator in self.config.get('additional_indicators', []):
            try:
                if indicator.lower() == 'sma':
                    sma = df['Close'].rolling(window=20).mean()
                    apds.append(mpf.make_addplot(sma, color='blue', linestyle='--', label='SMA(20)'))
                elif indicator.lower() == 'rsi':
                    rsi = self._calculate_rsi(df['Close'])
                    apds.append(mpf.make_addplot(rsi, panel=1, color='purple', ylabel='RSI', label='RSI(14)'))
                elif indicator.lower() == 'bollinger':
                    bbands = self._calculate_bollinger_bands(df['Close'])
                    apds.append(mpf.make_addplot(bbands['upper'], color='green', linestyle='--', label='BB Upper'))
                    apds.append(mpf.make_addplot(bbands['lower'], color='red', linestyle='--', label='BB Lower'))
                else:
                    logger.warning(f"Unsupported indicator: {indicator}")
            except Exception as e:
                logger.error(f"Error computing indicator {indicator}: {str(e)}")
        return apds

    def _calculate_rsi(self, series: pd.Series, period: int = 14) -> pd.Series:
        """Calculate Relative Strength Index (RSI)."""
        delta = series.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    def _calculate_bollinger_bands(self, series: pd.Series, period: int = 20, std_dev: float = 2) -> Dict:
        """Calculate Bollinger Bands."""
        sma = series.rolling(window=period).mean()
        std = series.rolling(window=period).std()
        return {
            'upper': sma + std * std_dev,
            'lower': sma - std * std_dev,
            'middle': sma
        }

    def plot(
        self,
        df: pd.DataFrame,
        apds: Optional[List[Any]] = None,
        custom_config: Optional[Dict[str, Any]] = None
    ) -> None:
        """
        Plot financial chart with OHLCV data and optional additional plots.

        Args:
            df (pd.DataFrame): DataFrame with OHLCV data and datetime index.
            apds (List[Any], optional): List of additional plots (mpf.make_addplot).
            custom_config (Dict[str, Any], optional): Override default configuration for this plot.

        Raises:
            ValueError: If DataFrame is invalid or configuration is incorrect.
        """
        start_time = datetime.now()
        chart_id = str(uuid4())
        logger.info(f"Generating chart {chart_id} with {len(df)} data points")

        try:
            # Validate input DataFrame
            self._validate_dataframe(df)

            # Merge custom config with instance config
            plot_config = self.config.copy()
            if custom_config:
                plot_config.update(custom_config)
                self._validate_config()

            # Handle additional plots
            all_apds = self._compute_indicators(df)  # Auto-computed indicators
            if apds is not None:  # User-provided plots
                if not isinstance(apds, list):
                    apds = [apds]
                all_apds.extend(apds)

            # Plot the chart
            mpf.plot(
                df,
                type=plot_config['chart_type'],
                style=plot_config['style'],
                addplot=all_apds if all_apds else None,  # Fix for None error
                volume=plot_config['volume'],
                volume_panel=plot_config['volume_panel'],
                title=plot_config['title'],
                ylabel=plot_config['ylabel'],
                ylabel_lower=plot_config['ylabel_lower'],
                xlabel=plot_config['xlabel'],
                figratio=plot_config['figratio'],
                panel_ratios=plot_config['panel_ratios'],
                tight_layout=True,
                datetime_format=plot_config['datetime_format'],
                xrotation=plot_config['xrotation'],
                warn_too_much_data=plot_config['warn_too_much_data'],
                update_width_config=plot_config['width_config']
            )
            logger.info(f"Chart {chart_id} generated successfully in {(datetime.now() - start_time).total_seconds():.2f}s")

        except Exception as e:
            logger.error(f"Failed to generate chart {chart_id}: {str(e)}")
            raise


In [ ]:
if 1 = 0:
    # Example usage
    if __name__ == "__main__":
        # Sample DataFrame
        data = {
            'Date': pd.date_range(start='2025-01-01', periods=100, freq='D'),
            'Open': [100 + i*0.1 for i in range(100)],
            'High': [101 + i*0.1 for i in range(100)],
            'Low': [99 + i*0.1 for i in range(100)],
            'Close': [100 + i*0.1 for i in range(100)],
            'Volume': [1000 + i*10 for i in range(100)]
        }
        df = pd.DataFrame(data).set_index('Date')

        # Initialize Chartter with custom indicators
        chartter = Chartter(config={'additional_indicators': ['sma', 'rsi', 'bollinger']})

        # Plot with additional custom plot
        custom_plot = mpf.make_addplot(df['Close'].rolling(10).mean(), color='orange', label='SMA(10)')
        chartter.plot(df, apds=[custom_plot], custom_config={'title': 'Enterprise Stock Chart'})

In [ ]:
chartter(ohlc_4H.iloc[100:251])